### Quickstart

In [13]:
import vectorbt as vbt
import datetime

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 7)

btc_price = vbt.YFData.download(
        ["BTC-USD","ETH-USD","XMR-USD","ADA-USD"],
        # "BTC-USD",
        interval="1h",
        start = start_date,
        end = end_date,
        missing_index='drop').get("Close")

# print(btc_price)

rsi = vbt.RSI.run(btc_price, window =[7,14,21])

entries = rsi.rsi_crossed_below(30)
exits = rsi.rsi_crossed_above(70)

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

# pf.plot().show()

print(pf.total_return())



rsi_window  symbol 
7           BTC-USD   -0.014743
            ETH-USD    0.030364
            XMR-USD    0.059616
            ADA-USD    0.050552
14          BTC-USD    0.021191
            ETH-USD    0.023580
            XMR-USD    0.048115
            ADA-USD    0.070416
21          BTC-USD    0.000000
            ETH-USD    0.000000
            XMR-USD    0.000000
            ADA-USD    0.034160
Name: total_return, dtype: float64


### Custom Indicators

In [14]:

import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime

end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)


btc_price = vbt.YFData.download(
        ["BTC-USD","ETH-USD"],
        missing_index='drop',
        start=start_time,
        end=end_time,
        interval="1m").get("Close")

def custom_indicator(close, rsi_window = 14, ma_window = 50):
    close_5m = close.resample("5T").last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    rsi, _ = rsi.align(close, 
            broadcast_axis=0,
            method='ffill',
            join='right')

    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where( rsi > 70, -1, 0)
    trend = np.where( (rsi < 30) & (close < ma), 1, trend)
    return trend

ind = vbt.IndicatorFactory(
        class_name = "Combination",
        short_name = "comb",
        input_names = ["close"],
        param_names = ["rsi_window", "ma_window"],
        output_names = ["value"]
        ).from_apply_func(
                custom_indicator,
                rsi_window = 14,
                ma_window = 50,
                keep_pd=True
                )

res = ind.run(
        btc_price,
        rsi_window = 21,
        ma_window = 50
        )

print(res.value.to_string())

entries = res.value == 1.0
exits = res.value == -1.0

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

print(pf.total_return())




/opt/anaconda3/lib/python3.12/site-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/3436234901.py:18: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/3436234901.py:20: FutureWarning:

The 'method', 'limit', and 'fill_axis' keywords in DataFrame.align are deprecated and will be removed in a future version. Call fillna directly on the returned objects instead.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/3436234901.py:20: FutureWarning:

The 'broadcast_axis' keyword in DataFrame.align is deprecated and will be removed in a future version.



comb_rsi_window                21        
comb_ma_window                 50        
symbol                    BTC-USD ETH-USD
Datetime                                 
2025-03-22 18:29:00+00:00       0       0
2025-03-22 18:30:00+00:00       0       0
2025-03-22 18:32:00+00:00       0       0
2025-03-22 18:35:00+00:00       0       0
2025-03-22 18:36:00+00:00       0       0
2025-03-22 18:38:00+00:00       0       0
2025-03-22 18:40:00+00:00       0       0
2025-03-22 18:41:00+00:00       0       0
2025-03-22 18:43:00+00:00       0       0
2025-03-22 18:45:00+00:00       0       0
2025-03-22 18:48:00+00:00       0       0
2025-03-22 18:50:00+00:00       0       0
2025-03-22 18:51:00+00:00       0       0
2025-03-22 18:53:00+00:00       0       0
2025-03-22 18:55:00+00:00       0       0
2025-03-22 18:56:00+00:00       0       0
2025-03-22 18:57:00+00:00       0       0
2025-03-22 19:03:00+00:00       0       0
2025-03-22 19:05:00+00:00       0       0
2025-03-22 19:08:00+00:00       0 

### Parameter Optimization

In [15]:

import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime

end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)


btc_price = vbt.YFData.download(
        ["BTC-USD","ETH-USD"],
        missing_index='drop',
        start=start_time,
        end=end_time,
        interval="1m").get("Close")

def custom_indicator(close, rsi_window = 14, ma_window = 50, entry = 30, exit = 70):
    close_5m = close.resample("5T").last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    rsi, _ = rsi.align(close, 
            broadcast_axis=0,
            method='ffill',
            join='right')

    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where( rsi > exit, -1, 0)
    trend = np.where( (rsi < entry) & (close < ma), 1, trend)
    return trend

ind = vbt.IndicatorFactory(
        class_name = "Combination",
        short_name = "comb",
        input_names = ["close"],
        param_names = ["rsi_window", "ma_window","entry","exit"],
        output_names = ["value"]
        ).from_apply_func(
                custom_indicator,
                rsi_window = 14,
                ma_window = 50,
                entry = 30,
                exit = 70,
                keep_pd=True
                )

res = ind.run(
        btc_price,
        rsi_window = np.arange(10,40,step=3,dtype=int),
        #ma_window = np.arange(20,200,step=20,dtype=int),
        entry = np.arange(10,40,step=4,dtype=int),
        exit = np.arange(60,85,step=4,dtype=int),
        param_product = True
        )

#print(res.value.to_string())

entries = res.value == 1.0
exits = res.value == -1.0

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

returns = pf.total_return()

#returns = returns[ returns.index.isin(["BTC-USD"], level="symbol")]

#returns = returns.groupby(level=["comb_exit","comb_entry","symbol"]).mean()

print(returns.to_string())

print(returns.max())
print(returns.idxmax())

"""comb_rsi_window  comb_ma_window"""

fig = returns.vbt.volume(
        x_level = "comb_rsi_window",
        y_level = "comb_entry",
        z_level = "comb_exit",
        slider_level = "symbol",
        )
fig.show()



/opt/anaconda3/lib/python3.12/site-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/4149004350.py:18: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/4149004350.py:20: FutureWarning:

The 'method', 'limit', and 'fill_axis' keywords in DataFrame.align are deprecated and will be removed in a future version. Call fillna directly on the returned objects instead.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/4149004350.py:20: FutureWarning:

The 'broadcast_axis' keyword in DataFrame.align is deprecated and will be removed in a future version.

/var/folders/yz/cl0zfhdd5g780lvqbcbjwglr0000gn/T/ipykernel_79635/4149004350.py:18: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

/v

comb_rsi_window  comb_entry  comb_exit  symbol 
10               10          60         BTC-USD    0.000000
                                        ETH-USD    0.003047
                             64         BTC-USD    0.000000
                                        ETH-USD    0.003047
                             68         BTC-USD    0.000000
                                        ETH-USD    0.003047
                             72         BTC-USD    0.000000
                                        ETH-USD    0.002883
                             76         BTC-USD    0.000000
                                        ETH-USD    0.002529
                             80         BTC-USD    0.000000
                                        ETH-USD    0.002529
                             84         BTC-USD    0.000000
                                        ETH-USD    0.002857
                 14          60         BTC-USD    0.001425
                                        ETH-USD    0

### Optimization Techniques

In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime
import talib
from numba import njit

end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)


btc_price = pd.read_csv("data.csv")
btc_price["Datetime"] = pd.to_datetime(btc_price["Datetime"])
btc_price.set_index("Datetime", inplace=True)

print(btc_price)

RSI = vbt.IndicatorFactory.from_talib('RSI')


@njit
def produce_signal(rsi, entry, exit):
    trend = np.where( rsi > exit, -1, 0)
    trend = np.where( (rsi < entry) , 1, trend)
    return trend

def custom_indicator(close, rsi_window = 14, entry = 30, exit = 70):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    return produce_signal(rsi, entry, exit)

ind = vbt.IndicatorFactory(
        class_name = "Combination",
        short_name = "comb",
        input_names = ["close"],
        param_names = ["rsi_window","entry","exit"],
        output_names = ["value"]
        ).from_apply_func(
                custom_indicator,
                rsi_window = 14,
                entry = 30,
                exit = 70,
                )

rsi_windows = np.arange(10,40,step=1,dtype=int)
entries = np.arange(10,40,step=1,dtype=int)

master_returns = []

for window in rsi_windows:
    res = ind.run(
            btc_price,
            rsi_window = window,
            entry = entry,
            exit = np.arange(60,85,step=1,dtype=int),
            param_product = True
            )
    entries = res.value == 1.0
    exits = res.value == -1.0
    pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
    master_returns.append(pf.total_return())

print(master_returns)

returns = pd.concat(master_returns)

#returns = returns[ returns.index.isin(["BTC-USD"], level="symbol")]

#returns = returns.groupby(level=["comb_exit","comb_entry","symbol"]).mean()

print(returns.to_string())

print(returns.max())
print(returns.idxmax())



### Graphing

In [17]:
import vectorbt as vbt
import datetime

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 3)

vbt.settings.set_theme("seaborn")
vbt.settings['plotting']['layout']['width'] = 1200
vbt.settings['plotting']['layout']['height'] = 600

btc_price = vbt.YFData.download(
        ["BTC-USD"],
        interval="1m",
        start = start_date,
        end = end_date,
        missing_index='drop').get("Close")

print(btc_price)

fast_ma = vbt.MA.run(btc_price, window =50)
slow_ma = vbt.MA.run(btc_price, window =200)

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

fig = pf.plot(subplots = [
    ('price', dict(
        title='Price',
        yaxis_kwargs=dict(title='Price')
        )),
    ('price', dict(
        title='Price',
        yaxis_kwargs=dict(title='Price')
        )),
    'orders',
    'trade_pnl',
    'cum_returns',
    'drawdowns'],
    make_subplots_kwargs=dict(rows=10, cols=2))

scatter = vbt.plotting.Scatter(
        data = btc_price,
        x_labels = btc_price.index,
        trace_names = ["Price"],
        trace_kwargs=dict(line=dict(color='red')),
        add_trace_kwargs=dict(row=1,col=1),
        fig=fig)

fast_ma_scatter = vbt.plotting.Scatter(
        data = fast_ma.ma,
        x_labels = fast_ma.ma.index,
        trace_names = ["Fast_ma"],
        trace_kwargs=dict(line=dict(color='green')),
        add_trace_kwargs=dict(row=1,col=2),
        fig=fig)

slow_ma_scatter = vbt.plotting.Scatter(
        data = slow_ma.ma,
        x_labels = slow_ma.ma.index,
        trace_names = ["slow_ma"],
        trace_kwargs=dict(line=dict(color='blue')),
        add_trace_kwargs=dict(row=1,col=2),
        fig=fig)


entries_plot = entries.vbt.signals.plot_as_entry_markers(
        slow_ma.ma,
        add_trace_kwargs=dict(row=1,col=2),
        fig=fig)

exits_plot = exits.vbt.signals.plot_as_exit_markers(
        slow_ma.ma,
        add_trace_kwargs=dict(row=1,col=2),
        fig=fig)

fig.add_hline( y= 38000,
                line_color="#FFFFFF",
                row=1,
                col = 2,
                line_width=20)

"""
fig = btc_price.vbt.plot(trace_kwargs=dict(name='Price',line=dict(color='red')))
fig = fast_ma.ma.vbt.plot(trace_kwargs=dict(name='Fast_ma',line=dict(color='blue')), fig=fig)
fig = slow_ma.ma.vbt.plot(trace_kwargs=dict(name='Slow_ma',line=dict(color='green')), fig=fig)
fig = entries.vbt.signals.plot_as_entry_markers(btc_price, fig=fig)
fig = exits.vbt.signals.plot_as_exit_markers(btc_price, fig=fig)
"""

fig.show()


Datetime
2025-03-21 22:26:00+00:00    84261.281250
2025-03-21 22:27:00+00:00    84270.132812
2025-03-21 22:28:00+00:00    84263.914062
2025-03-21 22:30:00+00:00    84241.343750
2025-03-21 22:32:00+00:00    84181.453125
                                 ...     
2025-03-24 22:15:00+00:00    87834.937500
2025-03-24 22:17:00+00:00    87725.023438
2025-03-24 22:18:00+00:00    87658.468750
2025-03-24 22:19:00+00:00    87614.703125
2025-03-24 22:21:00+00:00    87628.171875
Name: Close, Length: 2785, dtype: float64


### Order Types

In [2]:
import vectorbt as vbt
import datetime

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 1)

btc_price = vbt.YFData.download(
        "BTC-USD",
        interval="1m",
        start = start_date,
        end = end_date,
        missing_index='drop').get("Close")

print(btc_price)

rsi = vbt.RSI.run(btc_price, window =21)

entries = rsi.rsi_crossed_below(30)
exits = rsi.rsi_crossed_above(70)

pf = vbt.Portfolio.from_signals(
        btc_price,
        entries=entries,
        exits=exit,
        short_exits=entries,
        short_entries=exits,
        upon_dir_conflict=vbt.portfolio.enums.DirectionConflictMode.Short,
        )

pf.plot().show()

#print(pf.total_return())



Datetime
2025-03-24 13:04:00+00:00    87238.757812
2025-03-24 13:06:00+00:00    87237.343750
2025-03-24 13:07:00+00:00    87253.828125
2025-03-24 13:09:00+00:00    87218.000000
2025-03-24 13:13:00+00:00    87209.093750
                                 ...     
2025-03-25 12:53:00+00:00    87163.023438
2025-03-25 12:54:00+00:00    87125.929688
2025-03-25 12:55:00+00:00    87122.875000
2025-03-25 12:58:00+00:00    87040.960938
2025-03-25 12:59:00+00:00    87069.406250
Name: Close, Length: 955, dtype: float64


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 0d, C)[0m
[0m[1mDuring: typing of argument at /opt/anaconda3/lib/python3.12/site-packages/vectorbt/portfolio/nb.py (2375)[0m
[1m
File "../../../../../../../opt/anaconda3/lib/python3.12/site-packages/vectorbt/portfolio/nb.py", line 2375:[0m
[1mdef dir_enex_signal_func_nb(c: SignalContext,
    <source elided>

[1m@njit
[0m[1m^[0m[0m

[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function ls_enex_signal_func_nb at 0x13d76a520>))[0m
[0m[1mDuring: typing of call at /opt/anaconda3/lib/python3.12/site-packages/vectorbt/portfolio/nb.py (2126)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function ls_enex_signal_func_nb at 0x13d76a520>))[0m
[0m[1mDuring: typing of call at /opt/anaconda3/lib/python3.12/site-packages/vectorbt/portfolio/nb.p

### Reality Check

In [18]:

import pandas as pd
import vectorbt as vbt
import numpy as np
import datetime

now = datetime.datetime.now()
before = now - datetime.timedelta(days=3)

btc_price = vbt.YFData.download(
        "BTC-USD",
        missing_index="drop",
        interval="1m",
        start=before.timestamp(),
        end=now.timestamp()).get("Close")


btc_price, range_indexes = btc_price.vbt.range_split(n=100, range_len=1440)

# RSI SECTION
def optimize_rsi(close, window, entry, exit):
    rsi = vbt.IndicatorFactory.from_talib("RSI").run(close, timeperiod=window).real
    return rsi < entry, rsi > exit

rsi_ind = vbt.IndicatorFactory(
        class_name="optimizeRsi",
        short_name="rsi",
        input_names=["close"],
        param_names=["window","entry","exit"],
        output_names=["entries","exits"]
        ).from_apply_func(
                optimize_rsi,
                window=14,
                entry=30,
                exit=70)


step_size =10
entries = np.arange(10,45, step=step_size, dtype =int)
exits = np.arange(55,95, step=step_size, dtype =int)
windows = np.arange(10,45, step=step_size, dtype =int)

rsi_res = rsi_ind.run(
        btc_price,
        window=14,
        entry=20,
        exit=80,
        )

rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits

rsi_exits.iloc[-1, :] = True

rsi_pf = vbt.Portfolio.from_signals(btc_price, rsi_entries, rsi_exits, freq="1T")

rsi_tot_returns = rsi_pf.total_return()

def random_signal(close):
    return np.random.randint(0,2, size=close.shape)

rand_ind = vbt.IndicatorFactory(
        class_name="Random",
        short_name="rand",
        input_names=["close"],
        output_names=["signal"]
        ).from_apply_func(
                random_signal)



rand_res = rand_ind.run(
        btc_price
        )

rand_entries = rand_res.signal == 1
rand_exits = rand_res.signal == 0

rand_exits.iloc[-1, :] = True

rand_pf = vbt.Portfolio.from_signals(btc_price, rand_entries, rand_exits, freq="1T")

rand_tot_returns = rand_pf.total_return()


df = pd.DataFrame({
    "rsi":list(rsi_tot_returns),
    "rand":list(rand_tot_returns)
    })

print(df.median())

box = vbt.plotting.Box(
        data = df,
        trace_names=["rsi","rand"])

box.fig.show()


rsi     0.008119
rand    0.012919
dtype: float64
